### Graph based line merging

In [ ]:
import geopandas as gpd
from shapely.geometry import MultiPolygon
import linegroupping as lg

In [ ]:
in_file = r"I:\Temp\centerline.shp"
out_file = r"I:\Temp\centerline_grouped.shp"

in_poly_file = r"I:\Temp\footprint_ground.shp"
out_poly_file = r"I:\Temp\footprint_ground_clean.shp"

# in_file = r"I:\Temp\centerline_S.shp"
# out_file = r"I:\Temp\centerline_grouped_S.shp"

# in_poly_file = r"I:\Temp\footprint_ground_S.shp"
# out_poly_file = r"I:\Temp\footprint_ground_clean_S.shp"

In [ ]:
data = gpd.read_file(in_file)
data = data[~data.geometry.isna() & ~data.geometry.is_empty]
data.reset_index(inplace=True, drop=True)

In [ ]:
lg_obj = lg.LineGroupping(data)

In [ ]:

data["group"] = lg_obj.groups  # skip if has group attribute

1. save original lines with group attributes for checkinng and editing
2. dissolve and generate footprint

### clip polygon and line
**line_merge and own merge tool runs after line clipped**

In [ ]:
in_poly = gpd.read_file(in_poly_file)

sindex_poly = in_poly.sindex
for i in lg_obj.vertex_of_concern:
    polys = []
    primary_lines = []
    cleanup_lines = []

    # retrieve primary lines
    for j in i.line_connected[0]:  # only one connected line is available
        primary_lines.append(i.get_line(j))

    for j in i.line_not_connected:  # only one connected line is available
        cleanup_lines.append({'idx_line': j, 'line': i.get_line(j)})

    idx = sindex_poly.query(i.vertex, predicate="within")
    if len(idx) == 0:
        continue

    polys = in_poly.loc[idx].geometry
    poly_cleanup = []
    poly_primary = []
    for i, p in polys.items():
        if p.contains(primary_lines[0]) or p.contains(primary_lines[1]):
            poly_primary.append(p)
        else:
            for line in cleanup_lines:
                if p.contains(line['line']):
                    line.update({'idx_poly': i, 'poly': p})
                    poly_cleanup.append(line)

    poly_primary = MultiPolygon(poly_primary)
    poly_cleanup = lg.cleanup_poly_and_line(poly_cleanup, poly_primary)
    # TODO: update all same lines inn VertexNode
    
    for p in poly_cleanup:
        print(p['idx_poly'], p['idx_line'])
        in_poly.at[p['idx_poly'], 'geometry'] = p['poly']
        data.at[p['idx_line'], 'geometry'] = p['line']

In [ ]:
in_poly.to_file(out_poly_file)
data.to_file(out_file)